In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory('glpk').executable = 'D:\download\glpk-4.65\w64'

In [3]:
#Read generators and demand data as two dataframes  Desktop717data
df_genData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="GeneratorsData",skiprows=1) #read data from generators tab and skip the first row
df_AverloadsData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="AverageloadData",skiprows=1)
df_PeakloadsData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="PeakLoadData",skiprows=1) 
df_linesData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="LinesData",skiprows=1) 

In [4]:
#define the size of the sets
NumGens=len(df_genData)
NumNodes=len(df_AverloadsData)
NumLines=len(df_linesData)
#T=len(df_demandData)
print("We read data for", NumGens, "power generating units")
print("We read data for", NumNodes, "nodes")
print("We read data for", NumLines, "transmission lines")

We read data for 28 power generating units
We read data for 3 nodes
We read data for 3 transmission lines


In [5]:
#Define Indexes
G=np.array([g for g in range(0,NumGens)])  
N=np.array([n for n in range(0,NumNodes)])
L=np.array([l for l in range(0,NumLines)])

In [6]:
df_genData.columns

Index(['GenName', 'Node', 'FixedCost', 'SDCost', 'SUCost', 'VarCost', 'MaxGen',
       'SpringMaxGen', 'MinGen', 'RampDown', 'RampSD', 'RampSU', 'RampUp',
       'MinUpTime', 'MinDownTime', 'ReqUp', 'ReqDown', 'InitialStatus',
       'InitialGen', 'Plant Name', 'Fuel Type', 'Average Emission Rate'],
      dtype='object')

In [7]:
##Declare the generator's parameters and assign them the data we read before
VarCost=df_genData.loc[:,'VarCost'].to_numpy()
PMax=df_genData.loc[:,'MaxGen'].to_numpy()
SpringPMax=df_genData.loc[:,'SpringMaxGen'].to_numpy()
PMin=df_genData.loc[:,'MinGen'].to_numpy()
GenNode=df_genData.loc[:,'Node'].to_numpy()

In [8]:
df_linesData.columns  #In the data we defined 3 lines as: L1-2, L2-3, and L3-1.
#We could have defined them in other way, but this is convenient for the KVL equation

Index(['LineName', 'NodeFrom', 'NodeTo', 'Reactance', 'Capacity'], dtype='object')

In [9]:
##Declare the transmission lines' parameters and assign them the data we read before
NodeFrom=df_linesData.loc[:,'NodeFrom'].to_numpy()
NodeTo=df_linesData.loc[:,'NodeTo'].to_numpy()
LineReactance=df_linesData.loc[:,'Reactance'].to_numpy()
LineCapacity=df_linesData.loc[:,'Capacity'].to_numpy()

In [11]:
df_PeakloadsData.columns

Index(['LoadName', 'Node', 'PWinterPeak', 'PWinterOffPeak', 'PSpringPeak',
       'PSpringOffPeak', 'PSummerPeak', 'PSummerOffPeak', 'PFallPeak',
       'PFallOffPeak'],
      dtype='object')

In [12]:
PLoadBus=df_PeakloadsData.loc[:,'Node'].to_numpy( )
PPeakWinterDemand=df_PeakloadsData.loc[:,'PWinterPeak'].to_numpy( )
POffPeakWinterDemand=df_PeakloadsData.loc[:,'PWinterOffPeak'].to_numpy( )
PPeakSummerDemand=df_PeakloadsData.loc[:,'PSummerPeak'].to_numpy( )
POffPeakSummerDemand=df_PeakloadsData.loc[:,'PSummerOffPeak'].to_numpy( )
PPeakFallDemand=df_PeakloadsData.loc[:,'PFallPeak'].to_numpy( )
POffPeakFallDemand=df_PeakloadsData.loc[:,'PFallOffPeak'].to_numpy( )
PPeakSpringDemand=df_PeakloadsData.loc[:,'PSpringPeak'].to_numpy( )
POffPeakSpringDemand=df_PeakloadsData.loc[:,'PSpringOffPeak'].to_numpy( )

In [13]:
POffPeakWinterDemand

array([ 5432.6 , 12223.35,  9507.05])

In [14]:
# define a function indicatorMatrix that creates indicator matrices to relate generators with their node
# and relate lines with the nodes
#dataRows is a column vector of NumRows rows. Each element says the column where this row should be a one in the final matrix
def IndicatorMatrix(NumRows, NumCols, dataRows):
    matrix= np.zeros((NumRows, NumCols),dtype=int)
    for i in range(0,NumRows):
        matrix[i,dataRows[i]-1]=1
    return matrix

In [15]:
#Create indicator matrices using the function defined above
#GeneratorInBus is a matrix of G rows and N columns that has 1 in the position g,n if generator g is AT node N, 0 otherwise
GeneratorInBus=IndicatorMatrix(NumGens,NumNodes,GenNode)
#LineFromBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l departs FROM node N, 0 otherwise
LineFromBus=IndicatorMatrix(NumLines,NumNodes,NodeFrom)
#LineToBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l arrives TO node N, 0 otherwise
LineToBus=IndicatorMatrix(NumLines,NumNodes,NodeTo)

In [16]:
GeneratorInBus

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [17]:
#UC MODEL of winter peak hour
def PeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            == PPeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m



In [18]:
PPeakWinterDemand

array([ 5930.4, 13343.4, 10378.2])

In [19]:
m=PeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 723801.512
  Upper bound: 723801.512
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04719424247741699
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [20]:
print('SOLUTION-winter peak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-winter peak
The total system cost is = $ 723801.512
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 1718.05 
generator  6 = 0.00 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = -141.00 
line  2 = -300.20 
line  3 = 441.20 
LMPs in $/MWh
Node  1 = 39.00
Node  2 = 39.00
Node  3 = 39.00


In [21]:
#UC MODEL of winter offpeak hour
def OffPeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == POffPeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m


In [22]:
m= OffPeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 627501.461
  Upper bound: 627501.461
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.041056156158447266
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [23]:
print('SOLUTION-winter offpeak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-winter offpeak
The total system cost is = $ 627501.461
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 1787.25 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = -348.42 
line  2 = 612.43 
line  3 = -264.02 
LMPs in $/MWh
Node  1 = 38.00
Node  2 = 38.00
Node  3 = 38.00


#AOffPeakWinterDemand=df_AverloadsData.loc[:,'AWinterOffPeak'].to_numpy( )
APeakSummerDemand=df_AverloadsData.loc[:,'ASummerPeak'].to_numpy( )
AOffPeakSummerDemand=df_AverloadsData.loc[:,'ASummerOffPeak'].to_numpy( )
APeakFallDemand=df_AverloadsData.loc[:,'AFallPeak'].to_numpy( )
AOffPeakFallDemand=df_AverloadsData.loc[:,'AFallOffPeak'].to_numpy( )
APeakSpringDemand=df_AverloadsData.loc[:,'ASpringPeak'].to_numpy( )
AOffPeakSpringDemand=df_AverloadsData.loc[:,'ASpringOffPeak'].to_numpy( )

In [24]:
#UC MODEL of summer peak hour
def PeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == PPeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [25]:
m= PeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 894525.669
  Upper bound: 894525.669
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05627799034118652
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [26]:
print('SOLUTION- summer peak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION- summer peak
The total system cost is = $ 894525.6689999999
Generation in MW
generator  1 = 1148.40 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 2119.00 
generator  6 = 763.20 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 583.20 
generator 12 = 920.00 
generator 13 = 1019.25 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 422.70 
line  2 = -479.00 
line  3 = 56.30 
LMPs in $/MWh
Node  1 = 46.00
Node  2 = 46.00
Node  3 = 46.00


In [27]:
#UC MODEL of summer offpeak hour
def OffPeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == POffPeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [28]:
m= OffPeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 787179.967
  Upper bound: 787179.967
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05594778060913086
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [29]:
print('SOLUTION- summer offpeak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION- summer offpeak
The total system cost is = $ 787179.9669999998
Generation in MW
generator  1 = 554.85 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 2119.00 
generator  6 = 0.00 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 583.20 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 366.60 
line  2 = -485.15 
line  3 = 118.55 
LMPs in $/MWh
Node  1 = 44.00
Node  2 = 44.00
Node  3 = 44.00


In [30]:
#UC MODEL of fall peak hour
def PeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == PPeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [31]:
m= PeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 853493.669
  Upper bound: 853493.669
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0487210750579834
# ----------------------------------------------------------
#   Solution Information
# -------------------------

In [32]:
print('SOLUTION-fall peak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-fall peak hour
The total system cost is = $ 853493.6689999999
Generation in MW
generator  1 = 1148.40 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 2119.00 
generator  6 = 763.20 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 583.20 
generator 12 = 920.00 
generator 13 = 127.25 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 645.70 
line  2 = -746.60 
line  3 = 100.90 
LMPs in $/MWh
Node  1 = 46.00
Node  2 = 46.00
Node  3 = 46.00


In [33]:
#UC MODEL of fall offpeak hour
def OffPeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == POffPeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [34]:
m= OffPeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 720096.512
  Upper bound: 720096.512
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.056114912033081055
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [35]:
print('SOLUTION- fall offpeak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION- fall offpeak hour
The total system cost is = $ 720096.512
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 1623.05 
generator  6 = 0.00 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = -148.92 
line  2 = -265.37 
line  3 = 414.28 
LMPs in $/MWh
Node  1 = 39.00
Node  2 = 39.00
Node  3 = 39.00


In [36]:
#UC MODEL of spring peak hour
def PeakSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= SpringPMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == PPeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m


In [37]:
m= PeakSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1058911.45
  Upper bound: 1058911.45
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06384062767028809
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [38]:
print('SOLUTION-spring peak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-spring peak hour
The total system cost is = $ 1058911.4499999997
Generation in MW
generator  1 = 1148.40 
generator  2 = 413.60 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 2119.00 
generator  6 = 763.20 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 583.20 
generator 12 = 920.00 
generator 13 = 1753.60 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 1205.00 
generator 22 = 1250.00 
generator 23 = 1400.00 
generator 24 = 1441.10 
generator 25 = 384.30 
generator 26 = 475.45 
generator 27 = 1250.00 
generator 28 = 796.55 
Flow on transmission lines in MW
line  1 = 703.70 
line  2 = 8.30 
line  3 = -712.00 
LMPs in $/MWh
Node  1 = 100.00
Node  2 = 100.00
Node  3 = 100.00


In [39]:
#UC MODEL of  spring peak hour
def PeakOffSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= SpringPMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == POffPeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [40]:
m= PeakOffSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 866218.3
  Upper bound: 866218.3
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07010221481323242
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [41]:
print('SOLUTION- spring offpeak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION- spring offpeak hour
The total system cost is = $ 866218.2999999997
Generation in MW
generator  1 = 1148.40 
generator  2 = 402.15 
generator  3 = 2491.20 
generator  4 = 1530.50 
generator  5 = 2119.00 
generator  6 = 763.20 
generator  7 = 2558.20 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 583.20 
generator 12 = 920.00 
generator 13 = 1753.60 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 1205.00 
generator 22 = 1250.00 
generator 23 = 1400.00 
generator 24 = 1441.10 
generator 25 = 384.30 
generator 26 = 475.45 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 377.23 
line  2 = -188.62 
line  3 = -188.62 
LMPs in $/MWh
Node  1 = 47.00
Node  2 = 47.00
Node  3 = 47.00
